In [355]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing as ps
import numpy as np
import datetime

Lo primero que haremos será enfocarnos en la codificación de los datos

In [356]:
df = pd.read_csv('../BD_Limpia.csv')

C:\Users\andre\AppData\Local\Temp\ipykernel_8876\3697060408.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../BD_Limpia.csv')


Para ello primero vamos a ver cuales columnas hay que codificar (Aquellas que no sean numericas)
Para esto tomaremos las que son tipo "object" ya que estas pueden ser strings, datetime, etc, el resto podrán ser flotantes, booleanos u otro tipo de valor que no es necesario codificar

In [357]:
df2 = df.copy()
j = 0
for i in [i for i in df.columns.tolist()]:
    if df[i].dtype == object:
        print(i)
        j+=1
print(j)

DESCRIPCION
Mes
Tipo Egreso (INTERNACION)
Paciente Modalidad Contrato
Paciente Regimen Afiliacion
Urg Fecha Ingreso
Urg Fecha Triaje
Urg Fecha Consulta F3
Profesional Especialidad
Dx Principal Tipo 
Dx Principal Cod
Dx Principal Cod.1
Dx Relacionado1 Cod
Dx Relacionado2 Cod 
Dx Relacionado3 Cod
Unidad
Grupo Edad
Paciente Sexo
Clasificacion 1 Triaje 
Enfermeria 
Ubicacion  Urgencias
Causa Externa  (Riesgo)
Modalidad Atencion
Ubicación Triaje
Tiempo Duracion Diligenciamiento F3
Triaje Re-clasificado
Grupo Poblacional
Pertenencia Etnica
ALTO COSTO
28


Ahora tomaremos uno por uno cada valor entregado aqui y se revisará su posible codificación

In [358]:
df['DESCRIPCION'].unique()

array(['URGENCIAS ADULTO', 'URGENCIAS PEDIATRICAS'], dtype=object)

In [359]:
df2['DESCRIPCION'].replace({'URGENCIAS ADULTO': 1, 'URGENCIAS PEDIATRICAS': 0}, inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_8876\1354844995.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['DESCRIPCION'].replace({'URGENCIAS ADULTO': 1, 'URGENCIAS PEDIATRICAS': 0}, inplace=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_8876\1354844995.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downc

In [360]:
df['Tipo Egreso (INTERNACION)'].unique()

array(['No hospitalizacion', 'Hospitalizacion'], dtype=object)

In [361]:
df2['Tipo Egreso (INTERNACION)'].replace({'No hospitalizacion': 1, 'Hospitalizacion': 0}, inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_8876\673998892.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['Tipo Egreso (INTERNACION)'].replace({'No hospitalizacion': 1, 'Hospitalizacion': 0}, inplace=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_8876\673998892.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_sile

In [362]:
df["Modalidad Atencion"].unique() # Remitido y remitidos son lo mismo

array(['URGENCIAS', 'REMITIDO', 'REMITIDOS'], dtype=object)

In [363]:
df2["Modalidad Atencion"].replace({'URGENCIAS': 1, 'REMITIDO': 0, 'REMITIDOS': 0}, inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_8876\3878965590.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2["Modalidad Atencion"].replace({'URGENCIAS': 1, 'REMITIDO': 0, 'REMITIDOS': 0}, inplace=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_8876\3878965590.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_do

In [364]:
len(df["Dx Principal Cod"].unique()) 
# Al ser tan grande, se codificará con un valor diferente para cada único, 
# pues algunos podrian presentarse mas o tener mas peso

2434

In [365]:
encoder=ps.LabelEncoder()
df2["Dx Principal Cod"]=encoder.fit_transform(df2["Dx Principal Cod"])

Ahora esto nos ha entregado las 2400 enfermedades en numeros en vez de códigos, se evaluará posteriormente la viabilidad de hacerlo de esta manera, ya que supone un peso importante en la matriz de correlaciones.

Para los diagnósticos secundarios se pondrá un 1 en caso de existir uno, y un 0 en caso de que no, se sumarán posteriormente en una única variable.

In [366]:
varabiesl = ['Dx Relacionado3 Cod', 'Dx Relacionado2 Cod ', 'Dx Relacionado1 Cod']

def ifnostringis0(aver):
    if isinstance(aver,str):
        return 1
    else:
        return 0
df2[varabiesl] = df2[varabiesl].applymap(ifnostringis0)
df2['Dx Relacionado3 Cod'].unique()
df2['DxRelacionados'] = df2['Dx Relacionado3 Cod'] + df2['Dx Relacionado1 Cod'] + df2['Dx Relacionado2 Cod ']
df2 = df2.drop(columns=['Dx Relacionado3 Cod', 'Dx Relacionado1 Cod', 'Dx Relacionado2 Cod ', 'Dx Principal Cod.1', "Profesional Especialidad"])
df2['DxRelacionados'].unique()

C:\Users\andre\AppData\Local\Temp\ipykernel_8876\2403177839.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2[varabiesl] = df2[varabiesl].applymap(ifnostringis0)


array([0, 1, 2, 3], dtype=int64)

In [367]:
df["Mes"].unique()

array(['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO',
       'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE'],
      dtype=object)

In [368]:
meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO','AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
codificacion = {j:i+1 for i,j in enumerate(meses)}
df2["Mes"].replace(codificacion, inplace = True)

C:\Users\andre\AppData\Local\Temp\ipykernel_8876\399413001.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2["Mes"].replace(codificacion, inplace = True)
C:\Users\andre\AppData\Local\Temp\ipykernel_8876\399413001.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2["Mes"].replace(codificaci

In [369]:
confecha = ["Urg Fecha Ingreso", 'Urg Fecha Triaje', 'Urg Fecha Consulta F3']
def todoASegundos(datetaim):
    try:
        datetaim = datetaim[11:19].split(':')
        horas = int(datetaim[0])+int(datetaim[1])/60
        return horas
    except TypeError:
        return 0

df2[confecha] = df2[confecha].applymap(todoASegundos)
df2[confecha]

C:\Users\andre\AppData\Local\Temp\ipykernel_8876\919806519.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2[confecha] = df2[confecha].applymap(todoASegundos)


,Urg Fecha Ingreso,Urg Fecha Triaje,Urg Fecha Consulta F3
0,14.600000,14.383333,15.316667
1,18.533333,18.400000,19.716667
2,11.316667,0.000000,17.083333
3,10.666667,10.283333,12.083333
4,20.066667,20.000000,20.283333
...,...,...,...
36663,0.000000,3.000000,3.600000
36664,0.000000,2.166667,3.400000
36665,0.000000,9.383333,9.950000
36666,0.000000,11.366667,12.166667


In [370]:
df["Dx Principal Tipo "].unique()

array(['NUEVO', 'REPETIDO'], dtype=object)

In [371]:
df2["Dx Principal Tipo "].replace({'NUEVO': 1, 'REPETIDO': 0}, inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_8876\1481903230.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2["Dx Principal Tipo "].replace({'NUEVO': 1, 'REPETIDO': 0}, inplace=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_8876\1481903230.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
 

In [372]:
df["ALTO COSTO"].unique() # PARA ESTE SE PONDRÁ 1 SI ES STRING Y 0 SI NO, 
#YA QUE SI ES STRING ES UNA ENFERMEDADE DE ALTO COSTO
df2["ALTO COSTO"] = df2["ALTO COSTO"].apply(ifnostringis0)

In [373]:
df["Paciente Sexo"].unique()

array(['FEMENINO', 'MASCULINO'], dtype=object)

In [374]:
df2["Paciente Sexo"].replace({'FEMENINO': 0, 'MASCULINO': 1}, inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_8876\928752363.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2["Paciente Sexo"].replace({'FEMENINO': 0, 'MASCULINO': 1}, inplace=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_8876\928752363.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df

Ahora se ejecutarán todas las que se hagan con one-hot-encoding

In [375]:
dfTemp = df2[(df2["Unidad"] == 'MESES') | (df2["Unidad"] == 'DÍAS')]
listaInfantes = dfTemp.index.tolist()
df2.loc[listaInfantes, 'Grupo Edad'] = '0' #No tienen ni un año
df2["Grupo Edad"].unique()

array(['60-64', '75-79', '70-74', '5-9', '55-59', '65-69', '> 90',
       '20-24', '50-54', '80-84', '35-39', '85-89', '30-34', '45-49',
       '15-19', '1-4', '40-44', '25-29', '0', '10-14', nan], dtype=object)

In [376]:
df2 = pd.get_dummies(df2, columns=["Paciente Modalidad Contrato", "Paciente Regimen Afiliacion", "Grupo Edad"])
df2

,DESCRIPCION,Mes,Tipo Egreso (INTERNACION),Paciente Entidad Responsable Pago,Urg Fecha Ingreso,Urg Fecha Triaje,Urg Fecha Consulta F3,Minutos (Camara - F3),Urg Demora1 Consulta (Minutos),Urg Demora Triaje (Minutos)\n,...,Grupo Edad_5-9,Grupo Edad_50-54,Grupo Edad_55-59,Grupo Edad_60-64,Grupo Edad_65-69,Grupo Edad_70-74,Grupo Edad_75-79,Grupo Edad_80-84,Grupo Edad_85-89,Grupo Edad_> 90
0,1,1,1,5,14.600000,14.383333,15.316667,65.133333,35.133333,9.466667,...,False,False,False,True,False,False,False,False,False,False
1,1,1,1,65,18.533333,18.400000,19.716667,82.050000,52.050000,3.216667,...,False,False,False,True,False,False,False,False,False,False
2,1,1,0,48,11.316667,0.000000,17.083333,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,True,False,False,False
3,1,1,0,19,10.666667,10.283333,12.083333,116.850000,86.850000,8.600000,...,False,False,False,False,False,True,False,False,False,False
4,0,1,1,48,20.066667,20.000000,20.283333,21.450000,21.450000,5.083333,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36663,1,12,1,65,0.000000,3.000000,3.600000,50.133333,20.133333,13.883333,...,False,False,False,False,False,False,False,False,False,False
36664,1,12,0,4,0.000000,2.166667,3.400000,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,True,False
36665,1,12,1,18,0.000000,9.383333,9.950000,40.900000,10.900000,6.950000,...,False,True,False,False,False,False,False,False,False,False
36666,1,12,1,18,0.000000,11.366667,12.166667,0.000000,0.000000,0.000000,...,False,True,False,False,False,False,False,False,False,False


In [377]:
j = 0
for i in [i for i in df2.columns.tolist()]:
    if df2[i].dtype == object:
        print(i)
        j+=1
print(j)

Unidad
Clasificacion 1 Triaje 
Enfermeria 
Ubicacion  Urgencias
Causa Externa  (Riesgo)
Ubicación Triaje
Tiempo Duracion Diligenciamiento F3
Triaje Re-clasificado
Grupo Poblacional
Pertenencia Etnica
9


In [378]:
df["Unidad"].unique()

array(['AÑOS', 'MESES', 'DÍAS'], dtype=object)

In [379]:
df2 = df2.drop(columns=['Unidad'])

In [380]:
df2["Clasificacion 1 Triaje \nEnfermeria "].unique()

array(['TRIAJE 3', 'NO APLICA', 'TRIAJE 2', 'TRIAJE 1', 'TRIAJE 4',
       'TRIAJE 5'], dtype=object)

Para esta variables es conveniente hacer un onehot enconding

In [381]:

df2 = pd.get_dummies(df2, columns=["Clasificacion 1 Triaje \nEnfermeria "])
df2


,DESCRIPCION,Mes,Tipo Egreso (INTERNACION),Paciente Entidad Responsable Pago,Urg Fecha Ingreso,Urg Fecha Triaje,Urg Fecha Consulta F3,Minutos (Camara - F3),Urg Demora1 Consulta (Minutos),Urg Demora Triaje (Minutos)\n,...,Grupo Edad_75-79,Grupo Edad_80-84,Grupo Edad_85-89,Grupo Edad_> 90,Clasificacion 1 Triaje \nEnfermeria _NO APLICA,Clasificacion 1 Triaje \nEnfermeria _TRIAJE 1,Clasificacion 1 Triaje \nEnfermeria _TRIAJE 2,Clasificacion 1 Triaje \nEnfermeria _TRIAJE 3,Clasificacion 1 Triaje \nEnfermeria _TRIAJE 4,Clasificacion 1 Triaje \nEnfermeria _TRIAJE 5
0,1,1,1,5,14.600000,14.383333,15.316667,65.133333,35.133333,9.466667,...,False,False,False,False,False,False,False,True,False,False
1,1,1,1,65,18.533333,18.400000,19.716667,82.050000,52.050000,3.216667,...,False,False,False,False,False,False,False,True,False,False
2,1,1,0,48,11.316667,0.000000,17.083333,0.000000,0.000000,0.000000,...,True,False,False,False,True,False,False,False,False,False
3,1,1,0,19,10.666667,10.283333,12.083333,116.850000,86.850000,8.600000,...,False,False,False,False,False,False,False,True,False,False
4,0,1,1,48,20.066667,20.000000,20.283333,21.450000,21.450000,5.083333,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36663,1,12,1,65,0.000000,3.000000,3.600000,50.133333,20.133333,13.883333,...,False,False,False,False,False,False,False,True,False,False
36664,1,12,0,4,0.000000,2.166667,3.400000,0.000000,0.000000,0.000000,...,False,False,True,False,False,False,False,True,False,False
36665,1,12,1,18,0.000000,9.383333,9.950000,40.900000,10.900000,6.950000,...,False,False,False,False,False,False,False,True,False,False
36666,1,12,1,18,0.000000,11.366667,12.166667,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,True,False,False


In [382]:
df["Ubicacion  Urgencias"].unique()

array(['OBSERVACION', 'ALTA', 'XPRESS', 'HOSPITALIZACION'], dtype=object)

In [383]:
df2 = df2.drop(columns=['Ubicacion  Urgencias'])

In [384]:
j = 0
for i in [i for i in df2.columns.tolist()]:
    if df2[i].dtype == object:
        print(i)
        j+=1
print(j)

Causa Externa  (Riesgo)
Ubicación Triaje
Tiempo Duracion Diligenciamiento F3
Triaje Re-clasificado
Grupo Poblacional
Pertenencia Etnica
6


In [385]:
df["Causa Externa  (Riesgo)"].unique()

array(['ENFERMEDAD GENERAL', 'ACCIDENTE DE TRANSITO',
       'ACCIDENTE DE TRABAJO', 'OTRO TIPO DE ACCIDENTE',
       'ACCIDENTE RÁBICO', 'ENFERMEDAD PROFESIONAL', 'OTRA',
       'LESIÓN POR AGRESIÓN', 'SOSPECHA ABUSO SEXUAL'], dtype=object)

In [386]:
df2 = pd.get_dummies(df2, columns=["Causa Externa  (Riesgo)"])
df2

,DESCRIPCION,Mes,Tipo Egreso (INTERNACION),Paciente Entidad Responsable Pago,Urg Fecha Ingreso,Urg Fecha Triaje,Urg Fecha Consulta F3,Minutos (Camara - F3),Urg Demora1 Consulta (Minutos),Urg Demora Triaje (Minutos)\n,...,Clasificacion 1 Triaje \nEnfermeria _TRIAJE 5,Causa Externa (Riesgo)_ACCIDENTE DE TRABAJO,Causa Externa (Riesgo)_ACCIDENTE DE TRANSITO,Causa Externa (Riesgo)_ACCIDENTE RÁBICO,Causa Externa (Riesgo)_ENFERMEDAD GENERAL,Causa Externa (Riesgo)_ENFERMEDAD PROFESIONAL,Causa Externa (Riesgo)_LESIÓN POR AGRESIÓN,Causa Externa (Riesgo)_OTRA,Causa Externa (Riesgo)_OTRO TIPO DE ACCIDENTE,Causa Externa (Riesgo)_SOSPECHA ABUSO SEXUAL
0,1,1,1,5,14.600000,14.383333,15.316667,65.133333,35.133333,9.466667,...,False,False,False,False,True,False,False,False,False,False
1,1,1,1,65,18.533333,18.400000,19.716667,82.050000,52.050000,3.216667,...,False,False,False,False,True,False,False,False,False,False
2,1,1,0,48,11.316667,0.000000,17.083333,0.000000,0.000000,0.000000,...,False,False,False,False,True,False,False,False,False,False
3,1,1,0,19,10.666667,10.283333,12.083333,116.850000,86.850000,8.600000,...,False,False,False,False,True,False,False,False,False,False
4,0,1,1,48,20.066667,20.000000,20.283333,21.450000,21.450000,5.083333,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36663,1,12,1,65,0.000000,3.000000,3.600000,50.133333,20.133333,13.883333,...,False,False,False,False,True,False,False,False,False,False
36664,1,12,0,4,0.000000,2.166667,3.400000,0.000000,0.000000,0.000000,...,False,False,False,False,True,False,False,False,False,False
36665,1,12,1,18,0.000000,9.383333,9.950000,40.900000,10.900000,6.950000,...,False,False,True,False,False,False,False,False,False,False
36666,1,12,1,18,0.000000,11.366667,12.166667,0.000000,0.000000,0.000000,...,False,False,True,False,False,False,False,False,False,False


In [387]:
df["Ubicación Triaje"].unique()

array(['TRAUMA', 'CONSULTORIO 1', 'XPRESS', 'CONSULTORIO PEDIATRIA',
       'UDEA', 'SAI ADULTO', 'CONSULTORIO 2', 'CONSULTORIO 1 MAGISTERIO',
       'CONSULTORIO ONCOLOGIA', 'CONSULTORIO 4', 'TAMIZAJE COVID',
       'CONSULTORIO 3'], dtype=object)

In [388]:
df2 = pd.get_dummies(df2, columns=["Ubicación Triaje"])
df2

,DESCRIPCION,Mes,Tipo Egreso (INTERNACION),Paciente Entidad Responsable Pago,Urg Fecha Ingreso,Urg Fecha Triaje,Urg Fecha Consulta F3,Minutos (Camara - F3),Urg Demora1 Consulta (Minutos),Urg Demora Triaje (Minutos)\n,...,Ubicación Triaje_CONSULTORIO 2,Ubicación Triaje_CONSULTORIO 3,Ubicación Triaje_CONSULTORIO 4,Ubicación Triaje_CONSULTORIO ONCOLOGIA,Ubicación Triaje_CONSULTORIO PEDIATRIA,Ubicación Triaje_SAI ADULTO,Ubicación Triaje_TAMIZAJE COVID,Ubicación Triaje_TRAUMA,Ubicación Triaje_UDEA,Ubicación Triaje_XPRESS
0,1,1,1,5,14.600000,14.383333,15.316667,65.133333,35.133333,9.466667,...,False,False,False,False,False,False,False,True,False,False
1,1,1,1,65,18.533333,18.400000,19.716667,82.050000,52.050000,3.216667,...,False,False,False,False,False,False,False,False,False,False
2,1,1,0,48,11.316667,0.000000,17.083333,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,True
3,1,1,0,19,10.666667,10.283333,12.083333,116.850000,86.850000,8.600000,...,False,False,False,False,False,False,False,False,False,False
4,0,1,1,48,20.066667,20.000000,20.283333,21.450000,21.450000,5.083333,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36663,1,12,1,65,0.000000,3.000000,3.600000,50.133333,20.133333,13.883333,...,False,False,False,False,False,False,False,False,False,False
36664,1,12,0,4,0.000000,2.166667,3.400000,0.000000,0.000000,0.000000,...,False,False,True,False,False,False,False,False,False,False
36665,1,12,1,18,0.000000,9.383333,9.950000,40.900000,10.900000,6.950000,...,False,False,False,False,False,False,False,True,False,False
36666,1,12,1,18,0.000000,11.366667,12.166667,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,True


In [389]:
df["Tiempo Duracion Diligenciamiento F3"]

0        18:48
1        18:55
2            0
3        30:32
4         5:41
         ...  
36663    22:27
36664      NaN
36665    10:48
36666      NaN
36667    38:17
Name: Tiempo Duracion Diligenciamiento F3, Length: 36668, dtype: object

LA variable anterior requiere un manejo diferente, al ser un tiempo expresado como "MM:SS" es decir minutos:segundos, y significar el tiempo de duración de cierto proceso, es conveniente convertirlo a segundo

In [390]:


def convertir_a_segunditos(tiempito):
    if pd.isnull(tiempito):
        return None
    elif ':' not in tiempito:
        if tiempito.strip() == '0':  # Verifica si el valor es '0' después de eliminar espacios en blanco
            return 0
        else:
            return None
    else:
        try:
            minutos, segs = map(int, tiempito.split(':'))
            return int(minutos * 60 + segs)
        except ValueError:
            return None

def normalizar_seg(tiempo):
    return tiempo/24

# aplicando
df2['Tiempo Duracion Diligenciamiento F3 en seg'] = df2['Tiempo Duracion Diligenciamiento F3'].apply(convertir_a_segunditos)
df2['Tiempo Duracion Diligenciamiento F3 en seg'] = df2['Tiempo Duracion Diligenciamiento F3 en seg'].apply(normalizar_seg)
df2.drop(columns=['Tiempo Duracion Diligenciamiento F3'], inplace=True)
df2



,DESCRIPCION,Mes,Tipo Egreso (INTERNACION),Paciente Entidad Responsable Pago,Urg Fecha Ingreso,Urg Fecha Triaje,Urg Fecha Consulta F3,Minutos (Camara - F3),Urg Demora1 Consulta (Minutos),Urg Demora Triaje (Minutos)\n,...,Ubicación Triaje_CONSULTORIO 3,Ubicación Triaje_CONSULTORIO 4,Ubicación Triaje_CONSULTORIO ONCOLOGIA,Ubicación Triaje_CONSULTORIO PEDIATRIA,Ubicación Triaje_SAI ADULTO,Ubicación Triaje_TAMIZAJE COVID,Ubicación Triaje_TRAUMA,Ubicación Triaje_UDEA,Ubicación Triaje_XPRESS,Tiempo Duracion Diligenciamiento F3 en seg
0,1,1,1,5,14.600000,14.383333,15.316667,65.133333,35.133333,9.466667,...,False,False,False,False,False,False,True,False,False,47.000000
1,1,1,1,65,18.533333,18.400000,19.716667,82.050000,52.050000,3.216667,...,False,False,False,False,False,False,False,False,False,47.291667
2,1,1,0,48,11.316667,0.000000,17.083333,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,True,0.000000
3,1,1,0,19,10.666667,10.283333,12.083333,116.850000,86.850000,8.600000,...,False,False,False,False,False,False,False,False,False,76.333333
4,0,1,1,48,20.066667,20.000000,20.283333,21.450000,21.450000,5.083333,...,False,False,False,True,False,False,False,False,False,14.208333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36663,1,12,1,65,0.000000,3.000000,3.600000,50.133333,20.133333,13.883333,...,False,False,False,False,False,False,False,False,False,56.125000
36664,1,12,0,4,0.000000,2.166667,3.400000,0.000000,0.000000,0.000000,...,False,True,False,False,False,False,False,False,False,NaN
36665,1,12,1,18,0.000000,9.383333,9.950000,40.900000,10.900000,6.950000,...,False,False,False,False,False,False,True,False,False,27.000000
36666,1,12,1,18,0.000000,11.366667,12.166667,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,True,NaN


In [391]:
print("Cantidad de nulos en la columna anterior: "+str(df2["Tiempo Duracion Diligenciamiento F3 en seg"].isnull().sum()))
print("Porcentaje del: "+str((df2["Tiempo Duracion Diligenciamiento F3 en seg"].isnull().sum() / df2["Tiempo Duracion Diligenciamiento F3 en seg"].shape[0]) * 100))

Cantidad de nulos en la columna anterior: 8632
Porcentaje del: 23.540962146831024


In [392]:
def reemplazar_nulos_con_promedio(df, columna):
    promedio = df[columna].mean(skipna=True)
    df[columna].fillna(promedio, inplace=True)
    
    return df

# Aplicando la función a la columna que deseas
df2 = reemplazar_nulos_con_promedio(df2, 'Tiempo Duracion Diligenciamiento F3 en seg')


C:\Users\andre\AppData\Local\Temp\ipykernel_8876\2439479038.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[columna].fillna(promedio, inplace=True)


In [393]:
print("Cantidad de nulos en la columna anterior: "+str(df2["Tiempo Duracion Diligenciamiento F3 en seg"].isnull().sum()))
print("Porcentaje del: "+str((df2["Tiempo Duracion Diligenciamiento F3 en seg"].isnull().sum() / df2["Tiempo Duracion Diligenciamiento F3 en seg"].shape[0]) * 100))

Cantidad de nulos en la columna anterior: 0
Porcentaje del: 0.0


In [394]:
df["Triaje Re-clasificado"].unique()

array(['NO', 'SI'], dtype=object)

In [395]:
df2['Triaje Re-clasificado'] = df2['Triaje Re-clasificado'].replace({'NO': 0, 'SI': 1})

C:\Users\andre\AppData\Local\Temp\ipykernel_8876\552999706.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2['Triaje Re-clasificado'] = df2['Triaje Re-clasificado'].replace({'NO': 0, 'SI': 1})


In [396]:

df2.drop(columns=['Grupo Poblacional', 'Pertenencia Etnica'], inplace=True)

In [397]:
j = 0
for i in [i for i in df2.columns.tolist()]:
    if df2[i].dtype == object:
        print(i)
        j+=1
print(j)

0


In [398]:
datos_nulos = df2.isnull().sum()
porcentaje_nulos = (datos_nulos / len(df2)) * 100

# Creamos una tablita para mejor visualizacion
resultados = pd.DataFrame({'Datos Nulos': datos_nulos, 'Porcentaje de Nulos': porcentaje_nulos})
print("Cantidad de datos nulos por columna y su porcentaje:")

def revisar_valores_diferentes_de_cero(df, columna):
    hay_diferentes_de_cero = (df[columna] != 0).any()
    if hay_diferentes_de_cero:
        print(f"Hay valores diferentes de cero en la columna '{columna}'.")
    else:
        print(f"No hay valores diferentes de cero en la columna '{columna}'.")
    
    return hay_diferentes_de_cero
print(resultados[resultados["Datos Nulos"]>0])
revisar_valores_diferentes_de_cero(resultados, 'Porcentaje de Nulos')

Cantidad de datos nulos por columna y su porcentaje:
              Datos Nulos  Porcentaje de Nulos
Hora Ingreso        27792            75.793608
Hay valores diferentes de cero en la columna 'Porcentaje de Nulos'.


True

In [399]:
def revisar_valores_diferentes_de_cero(df, columna):
    # Filtrar los valores diferentes de cero
    diferentes_de_cero = df[df[columna] != 0]
    cantidad_diferentes_de_cero = len(diferentes_de_cero)
    
    if cantidad_diferentes_de_cero > 0:
        print(f"Hay {cantidad_diferentes_de_cero} valores diferentes de cero en la columna '{columna}'.")
        print("Valores correspondientes en el índice 'DESCRIPCION':")
        print(diferentes_de_cero.index)
    else:
        print(f"No hay valores diferentes de cero en la columna '{columna}'.")
    
    return cantidad_diferentes_de_cero, diferentes_de_cero.index

# Ejemplo de uso
cantidad_diferentes_de_cero, indices_diferentes_de_cero = revisar_valores_diferentes_de_cero(resultados, 'Datos Nulos')




Hay 1 valores diferentes de cero en la columna 'Datos Nulos'.
Valores correspondientes en el índice 'DESCRIPCION':
Index(['Hora Ingreso'], dtype='object')


In [400]:
print("Cantidad de nulos en la columna anterior: "+str(df2["Hora Ingreso"].isnull().sum()))
print("Porcentaje del: "+str((df2["Hora Ingreso"].isnull().sum() / df2["Hora Ingreso"].shape[0]) * 100))

Cantidad de nulos en la columna anterior: 27792
Porcentaje del: 75.79360750518163


In [401]:
df2.drop(columns=["Hora Ingreso"], inplace=True)

In [402]:
df2

,DESCRIPCION,Mes,Tipo Egreso (INTERNACION),Paciente Entidad Responsable Pago,Urg Fecha Ingreso,Urg Fecha Triaje,Urg Fecha Consulta F3,Minutos (Camara - F3),Urg Demora1 Consulta (Minutos),Urg Demora Triaje (Minutos)\n,...,Ubicación Triaje_CONSULTORIO 3,Ubicación Triaje_CONSULTORIO 4,Ubicación Triaje_CONSULTORIO ONCOLOGIA,Ubicación Triaje_CONSULTORIO PEDIATRIA,Ubicación Triaje_SAI ADULTO,Ubicación Triaje_TAMIZAJE COVID,Ubicación Triaje_TRAUMA,Ubicación Triaje_UDEA,Ubicación Triaje_XPRESS,Tiempo Duracion Diligenciamiento F3 en seg
0,1,1,1,5,14.600000,14.383333,15.316667,65.133333,35.133333,9.466667,...,False,False,False,False,False,False,True,False,False,47.000000
1,1,1,1,65,18.533333,18.400000,19.716667,82.050000,52.050000,3.216667,...,False,False,False,False,False,False,False,False,False,47.291667
2,1,1,0,48,11.316667,0.000000,17.083333,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,True,0.000000
3,1,1,0,19,10.666667,10.283333,12.083333,116.850000,86.850000,8.600000,...,False,False,False,False,False,False,False,False,False,76.333333
4,0,1,1,48,20.066667,20.000000,20.283333,21.450000,21.450000,5.083333,...,False,False,False,True,False,False,False,False,False,14.208333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36663,1,12,1,65,0.000000,3.000000,3.600000,50.133333,20.133333,13.883333,...,False,False,False,False,False,False,False,False,False,56.125000
36664,1,12,0,4,0.000000,2.166667,3.400000,0.000000,0.000000,0.000000,...,False,True,False,False,False,False,False,False,False,41.285963
36665,1,12,1,18,0.000000,9.383333,9.950000,40.900000,10.900000,6.950000,...,False,False,False,False,False,False,True,False,False,27.000000
36666,1,12,1,18,0.000000,11.366667,12.166667,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,True,41.285963


In [403]:
for i in df2.dtypes:
    print(i)

int64
int64
int64
int64
float64
float64
float64
float64
float64
float64
float64
float64
int64
int32
int64
int64
float64
float64
int64
bool
int64
int64
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
bool
float64


In [404]:
def normalize_column(column):
    if not pd.api.types.is_bool_dtype(column):
        max_value = column.max(skipna=True)
        try:
            normalized_column = column/max_value
        except:
            print(column)
        return normalized_column
    return column

# Aplicar la función de normalización a cada columna del DataFrame
df2_normalized = df2.apply(normalize_column)

GUARDAMOS EL DATAFRAME

In [407]:

df2_normalized.to_csv('../BD_ParaModelos.csv', index=False)


SEGUIMOS CON EL PROCESO DE MODELADO

1. METRICAS DE EVALUACION:
Exactitud (Accuracy)
Precisión (Precision)
Recuperación (Recall o Sensibilidad)
Puntuación F1 (F1 Score)
AUC-ROC (Área bajo la curva de características operativas del receptor)
Matriz de confusión
Curva ROC (Receiver Operating Characteristic)

Principalmente se analizaran el accuracy y la matriz de confusion

2. METODOS SUPERVISADOS

In [ ]:
from sklearn.tree import DecisionTreeClassifier , plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score , KFold
from sklearn.metrics import classification_report


Para la metrica

In [ ]:
def train_test_evaluador(model_name, model, X_train, y_train, X_test, y_test):  
    
    y_pred= model.predict(X_test)
    
    accu = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred,squared=False)
    f1_score = f1_score= y_test.mean()
  
    
    result_df = pd.DataFrame(
        data=[[model_name, accu, precision, recall, f1_score]], 
        columns=["Model", 'ACCURACY', 'PRECISION', 'RECALL', 'F1 SCORE'])
    return result_df

COSAS A CORREGIR:

- LOS DIAGNOSTICO: LA CODIFICACIÓN DE LOS MISMO, LO DEL NUMERO DE DIAGNOSTICO SI SE INCLUYE O NO.
- NORMALIZAR: Urg Fecha Ingreso,	Urg Fecha Triaje,	Urg Fecha Consulta F3. PONERLA EN HORAS MEJOR EN VEZ DE SEGUNDOS, ASI LA NORMALIZACION NO ES TAN AGRESIVA.
- SOBRE LA VARIABLE DE LA EDAD, DE QUE HAY DIFERENTES UNIDADES, (TOCA CORRER DESDE EL PRINCIPIO PORQUE ELIMINÉ LA COLUMNA UNIDAD JEJE), TOCA TODO LO QUE ESTA EN MESES METERLO EN EL GRUPO 0-4
- NORMALIZAR TIEMPO DE DILIGENCIAMIENTO DEL F3 EN SEG
- VERIFICAR QUE PASA CON LA VARIABLES 'HORA INGRESO' DE DONDE SALIO Y POR QUE TIENE ESA CANTIDAD DE DATOS NULOS, COMO MENCIONÉ CREEEOOO QUE ESA ES LA HORA QUE TOMA LA CAMARA, PERO NO VEO RAZON PARA QUE TENGA TANTOS DATOS NULOS, HAY QUE VERIFICAR ESO EN EL NOTEBOOK ANTERIOR CREERIA.

ME AVISAS CUANDO ACABES, MIENTRAS IRÉ ORGANIZANDO EL FLUJO DE TRABAJO PARA PROBAR LOS DIFERENTES MODELOS
